In [2]:
import yfinance as yf
import pandas as pd
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import statsmodels
import seaborn as sns
import scipy
from scipy import stats
from statistics import mean


import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
from scipy.stats import probplot, moment
import requests

import graphviz
import pydotplus

import eikon as ek  # the Eikon Python wrapper package
import configparser as cp

import sklearn
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
# Load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


In [3]:
df = pd.read_csv("evaluation.csv")
df = df.dropna()
df

,Unnamed: 0,ExponentialMovingAverage,WeightedMovingAverage,StochasticSlow,MACDHisto,RSI,SmoothedMovingAverage,ATR,time,value
25,25,23.388462,23.031477,15.780879,-0.851644,35.237303,44.991152,0.868012,2012-04-20,10000000.0
26,26,23.273964,22.909754,12.558950,-0.846600,42.239855,44.716022,0.892440,2012-04-23,10000000.0
27,27,23.092121,22.720031,13.320477,-0.911975,36.193033,43.863723,0.954408,2012-04-24,10000000.0
28,28,22.918881,22.537046,17.591616,-0.958386,35.802760,43.057627,0.956665,2012-04-25,10000000.0
29,29,22.732813,22.340092,11.508232,-1.010949,33.891165,42.140981,0.921189,2012-04-26,10000000.0
...,...,...,...,...,...,...,...,...,...,...
2801,2801,46.282842,46.517231,62.745098,0.723529,57.229240,56.240661,1.258040,2023-05-03,10208800.0
2802,2802,46.267239,46.531046,47.944339,0.598694,49.313460,55.547941,1.326751,2023-05-04,10208800.0
2803,2803,46.342067,46.629508,42.694497,0.586601,54.917619,55.484908,1.318412,2023-05-05,10208800.0
2804,2804,46.477292,46.785908,46.679317,0.639046,58.574499,55.793867,1.301383,2023-05-08,10208800.0


In [4]:
df.columns

Index(['Unnamed: 0', 'ExponentialMovingAverage', 'WeightedMovingAverage',
       'StochasticSlow', 'MACDHisto', 'RSI', 'SmoothedMovingAverage', 'ATR',
       'time', 'value'],
      dtype='object')

In [5]:
feature_cols = ['ExponentialMovingAverage', 'WeightedMovingAverage','StochasticSlow', 'MACDHisto', 'RSI', 'SmoothedMovingAverage', 'ATR']
X = df[feature_cols].values # Features
y = df["value"].values # Target variable

df_2 = pd.read_csv("evaluation2.csv")
X_test = df_2[feature_cols].dropna().values
y_test = df_2["value"].dropna().values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X, y)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [8]:
print("Accuracy:",sklearn.metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [9]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=None)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('diabetes.png')
Image(graph.create_png())

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.534038 to fit

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.534038 to fit

